In [5]:
import numpy as np
import random
import os

In [6]:
def replace_at(s, index, char):
    return s[:index] + char + s[index+1:]

def string_val(s, index):
    return int(s[index])

def shuffle_array(arr):
    random.shuffle(arr)

class Maze:
    def __init__(self, args=None):
        if args is None:
            args = {}

        defaults = {
            'width': 20,
            'height': 20,
            'wallSize': 10,
            'entryType': '',
            'bias': '',
            'color': '#000000',
            'backgroundColor': '#FFFFFF',
            'solveColor': '#cc3737',
            'removeWalls': 0,
            'maxWallsRemove': 300,
            'maxMaze': 0,
            'maxCanvas': 0,
            'maxCanvasDimension': 0,
            'maxSolve': 0,
        }

        # Merge defaults with user args
        for k, v in args.items():
            defaults[k] = v

        self.width = int(defaults['width'])
        self.height = int(defaults['height'])
        self.wallSize = int(defaults['wallSize'])
        self.removeWalls = int(defaults['removeWalls'])
        self.bias = defaults['bias']
        self.color = defaults['color']
        self.backgroundColor = defaults['backgroundColor']
        self.solveColor = defaults['solveColor']
        self.maxMaze = int(defaults['maxMaze'])
        self.maxCanvas = int(defaults['maxCanvas'])
        self.maxCanvasDimension = int(defaults['maxCanvasDimension'])
        self.maxSolve = int(defaults['maxSolve'])
        self.maxWallsRemove = int(defaults['maxWallsRemove'])
        self.entryType = defaults['entryType']

        self.matrix = []
        self.wallsRemoved = 0
        self.entryNodes = self.getEntryNodes(self.entryType)

    def isValidSize(self):
        max_dim = self.maxCanvasDimension
        canvas_width = ((self.width * 2) + 1) * self.wallSize
        canvas_height = ((self.height * 2) + 1) * self.wallSize

        if max_dim and ((max_dim <= canvas_width) or (max_dim <= canvas_height)):
            return False

        if self.maxCanvas and (self.maxCanvas <= (canvas_width * canvas_height)):
            return False

        return True

    def getEntryNodes(self, access):
        # Returns start and end nodes depending on entry type, but we do not strictly need them for the array
        y = ((self.height * 2) + 1) - 2
        x = ((self.width * 2) + 1) - 2

        entryNodes = {}

        if access == 'diagonal':
            entryNodes['start'] = { 'x': 1, 'y': 1, 'gate': { 'x': 0, 'y': 1 } }
            entryNodes['end'] = { 'x': x, 'y': y, 'gate': { 'x': x + 1, 'y': y } }

        elif access == 'horizontal' or access == 'vertical':
            xy = (y if access == 'horizontal' else x)
            xy = ((xy - 1) // 2)
            even = (xy % 2 == 0)
            xy = xy + 1 if even else xy

            if access == 'horizontal':
                start_x = 1
                start_y = xy
                end_x = x
                end_y = (xy if even else xy + 2)
                startgate = { 'x': 0, 'y': start_y }
                endgate = { 'x': x + 1, 'y': end_y }

            else: # vertical
                start_x = xy
                start_y = 1
                end_x = (xy if even else xy + 2)
                end_y = y
                startgate = { 'x': start_x, 'y': 0 }
                endgate = { 'x': end_x, 'y': y + 1 }

            entryNodes['start'] = { 'x': start_x, 'y': start_y, 'gate': startgate }
            entryNodes['end'] = { 'x': end_x, 'y': end_y, 'gate': endgate }

        return entryNodes

    def generateNodes(self):
        count = self.width * self.height
        nodes = []
        for i in range(count):
            # visited, nswe
            nodes.append("01111")
        return nodes

    def getNeighbours(self, pos):
        return {
            'n': (pos - self.width if (pos - self.width) >= 0 else -1),
            's': (pos + self.width if (pos + self.width) < (self.width * self.height) else -1),
            'w': (pos - 1 if (pos % self.width) != 0 else -1),
            'e': (pos + 1 if ((pos + 1) % self.width) != 0 else -1),
        }

    def biasDirections(self, directions):
        horizontal = ('w' in directions or 'e' in directions)
        vertical = ('n' in directions or 's' in directions)

        if self.bias == 'horizontal' and horizontal:
            directions = [d for d in directions if d in ['w', 'e']]
        elif self.bias == 'vertical' and vertical:
            directions = [d for d in directions if d in ['n', 's']]

        return directions

    def parseMaze(self, nodes):
        mazeSize = len(nodes)
        positionIndex = { 'n': 1, 's': 2, 'w': 3, 'e': 4 }
        oppositeIndex = { 'n': 2, 's': 1, 'w': 4, 'e': 3 }

        if mazeSize == 0:
            return nodes

        max_steps = 0
        moveNodes = []
        visited = 0
        position = random.randint(0, len(nodes)-1)

        biasCount = 0
        biasFactor = 3
        if self.bias:
            if self.bias == 'horizontal':
                biasFactor = ( (self.width // 100) + 2 ) if (1 <= (self.width / 100)) else 3
            elif self.bias == 'vertical':
                biasFactor = ( (self.height // 100) + 2 ) if (1 <= (self.height / 100)) else 3

        # Set start node visited
        nodes[position] = replace_at(nodes[position], 0, '1')

        while visited < (mazeSize - 1):
            biasCount += 1
            max_steps += 1
            if self.maxMaze and (self.maxMaze < max_steps):
                print('Please use smaller maze dimensions (maxMaze exceeded)')
                return []

            next_nodes = self.getNeighbours(position)
            directions = [key for key in next_nodes if next_nodes[key] != -1 and string_val(nodes[next_nodes[key]], 0) == 0]

            if self.bias and (biasCount != biasFactor):
                directions = self.biasDirections(directions)
            else:
                biasCount = 0

            if directions:
                visited += 1

                if len(directions) > 1:
                    moveNodes.append(position)

                direction = random.choice(directions)

                # Update current position
                nodes[position] = replace_at(nodes[position], positionIndex[direction], '0')
                # Set new position
                position = next_nodes[direction]

                # Update next position
                nodes[position] = replace_at(nodes[position], oppositeIndex[direction], '0')
                nodes[position] = replace_at(nodes[position], 0, '1')
            else:
                if not moveNodes:
                    break
                position = moveNodes.pop()

        return nodes

    def getMatrix(self, nodes):
        mazeSize = self.width * self.height
        if len(nodes) != mazeSize:
            return

        self.matrix = []
        row1 = ''
        row2 = ''

        for i in range(mazeSize):
            if not row1:
                row1 = '1'
            if not row2:
                row2 = '1'

            if string_val(nodes[i], 1) == 1:
                # visited cell start
                row1 += '11'
                if string_val(nodes[i], 4) == 1:
                    row2 += '01'
                else:
                    row2 += '00'
            else:
                # not visited?
                hasAbove = (i - self.width >= 0)
                above = hasAbove and (string_val(nodes[i - self.width], 4) == 1)
                hasNext = (i + 1 < mazeSize)
                nxt = hasNext and (string_val(nodes[i + 1], 1) == 1)

                if string_val(nodes[i], 4) == 1:
                    row1 += '01'
                    row2 += '01'
                elif nxt or above:
                    row1 += '01'
                    row2 += '00'
                else:
                    row1 += '00'
                    row2 += '00'

            if ((i + 1) % self.width) == 0:
                self.matrix.append(row1)
                self.matrix.append(row2)
                row1 = ''
                row2 = ''

        # Add closing row
        self.matrix.append('1' * ((self.width * 2) + 1))

    def removeWall(self, row, index):
        evenRow = (row % 2 == 0)
        evenIndex = (index % 2 == 0)
        wall = string_val(self.matrix[row], index)

        if wall == 0:
            return False

        # The logic for removing walls checks conditions around the chosen wall
        # and tries to ensure it still forms a proper maze.
        # This logic is directly ported from the JS code.

        if not evenRow and evenIndex:
            # Uneven row and even column
            top_ok = (row - 2 > 0 and string_val(self.matrix[row - 2], index) == 1)
            bottom_ok = (row + 2 < len(self.matrix) and string_val(self.matrix[row + 2], index) == 1)

            if top_ok and bottom_ok:
                self.matrix[row] = replace_at(self.matrix[row], index, '0')
                return True
            elif not top_ok and bottom_ok:
                left = (string_val(self.matrix[row - 1], index - 1) == 1)
                right = (string_val(self.matrix[row - 1], index + 1) == 1)
                if left or right:
                    self.matrix[row] = replace_at(self.matrix[row], index, '0')
                    return True
            elif not bottom_ok and top_ok:
                left = (string_val(self.matrix[row + 1], index - 1) == 1)
                right = (string_val(self.matrix[row + 1], index + 1) == 1)
                if left or right:
                    self.matrix[row] = replace_at(self.matrix[row], index, '0')
                    return True

        elif evenRow and not evenIndex:
            # Even row and uneven column
            left_ok = (string_val(self.matrix[row], index - 2) == 1)
            right_ok = (string_val(self.matrix[row], index + 2) == 1)

            if left_ok and right_ok:
                self.matrix[row] = replace_at(self.matrix[row], index, '0')
                return True
            elif not left_ok and right_ok:
                top = (string_val(self.matrix[row - 1], index - 1) == 1)
                bottom = (string_val(self.matrix[row + 1], index - 1) == 1)
                if top or bottom:
                    self.matrix[row] = replace_at(self.matrix[row], index, '0')
                    return True
            elif not right_ok and left_ok:
                top = (string_val(self.matrix[row - 1], index + 1) == 1)
                bottom = (string_val(self.matrix[row + 1], index + 1) == 1)
                if top or bottom:
                    self.matrix[row] = replace_at(self.matrix[row], index, '0')
                    return True

        return False

    def removeMazeWalls(self):
        if self.removeWalls == 0 or not self.matrix:
            return

        minr = 1
        maxr = len(self.matrix) - 1
        maxTries = self.maxWallsRemove
        tries = 0

        while tries < maxTries:
            tries += 1
            if self.wallsRemoved >= self.removeWalls:
                break

            y = random.randint(minr, maxr)
            if y == maxr:
                y -= 1

            row = self.matrix[y]
            walls = []
            for i in range(len(row)):
                if i == 0 or i == (len(row)-1):
                    continue
                w = string_val(row, i)
                if w == 1:
                    walls.append(i)

            shuffle_array(walls)

            for w_index in walls:
                if self.removeWall(y, w_index):
                    self.wallsRemoved += 1
                    break

    def generate(self):
        if not self.isValidSize():
            print('Please use smaller maze dimensions (canvas too large)')
            self.matrix = []
            return

        nodes = self.generateNodes()
        nodes = self.parseMaze(nodes)
        if not nodes:
            self.matrix = []
            return
        self.getMatrix(nodes)
        self.removeMazeWalls()

    def to_numpy(self):
        # Convert the matrix of strings into a numpy array of ints
        arr = np.array([list(map(int, list(row))) for row in self.matrix])
        return arr

In [7]:
# Specify the dimensions here
height = 55
width = 55

maze = Maze({'height': (height/2), 'width': (width/2), 'removeWalls': 200})
while True:
    try:
        maze.generate()
        break
    except:
        pass
maze_array = maze.to_numpy()

os.makedirs("mazes", exist_ok=True)
filename = f'{maze_array.shape[0]}x{maze_array.shape[1]}_layout.npy'

np.save(os.path.join("mazes", filename), maze_array)